In [ ]:
# ========================================
# ✅ Step 1: Mount Google Drive
# ========================================
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ========================================
# ✅ Step 2: Set paths
# ========================================
import os

# Set your actual dataset folder (with class subfolders)
SRC_DATA = "/content/drive/MyDrive/CDAC_Wound_Dataset"  # ⬅️ change this folder name
DST_SPLIT = "/content/data_split"  # output split location

# Make sure dataset exists
if not os.path.exists(SRC_DATA):
    raise FileNotFoundError(f"Folder not found: {SRC_DATA}")


In [ ]:
# ========================================
# ✅ Step 3: Split Script (Stratified)
# ========================================
import random
import shutil

def stratified_split(
    src_dir,
    dst_dir,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    seed=42
):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1.0"
    random.seed(seed)
    os.makedirs(dst_dir, exist_ok=True)

    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(dst_dir, split), exist_ok=True)

    class_names = sorted([d for d in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir, d))])
    print("Found classes:", class_names)

    for cls in class_names:
        src_cls_path = os.path.join(src_dir, cls)
        images = [f for f in os.listdir(src_cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp'))]
        images.sort()
        random.shuffle(images)

        n = len(images)
        n_train = int(n * train_ratio)
        n_val = int(n * val_ratio)
        n_test = n - n_train - n_val

        splits = {
            'train': images[:n_train],
            'val': images[n_train:n_train + n_val],
            'test': images[n_train + n_val:]
        }

        for split, split_imgs in splits.items():
            split_cls_path = os.path.join(dst_dir, split, cls)
            os.makedirs(split_cls_path, exist_ok=True)

            for fname in split_imgs:
                src_file = os.path.join(src_cls_path, fname)
                dst_file = os.path.join(split_cls_path, fname)
                shutil.copy2(src_file, dst_file)

        print(f"Class '{cls}': {n} → train={n_train}, val={n_val}, test={n_test}")

    print(f"\n✅ Done. Split saved to: {dst_dir}")


In [ ]:
# ========================================
# ✅ Step 4: Run the Split
# ========================================
stratified_split(SRC_DATA, DST_SPLIT, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, seed=123)


Found classes: ['Abrasions', 'Bruises', 'Burns', 'Cut', 'Diabetic Wounds', 'Laseration', 'Normal', 'Pressure Wounds', 'Surgical Wounds', 'Venous Wounds', 'healthy']
Class 'Abrasions': 164 → train=131, val=16, test=17
Class 'Bruises': 242 → train=193, val=24, test=25
Class 'Burns': 134 → train=107, val=13, test=14
Class 'Cut': 100 → train=80, val=10, test=10
Class 'Diabetic Wounds': 462 → train=369, val=46, test=47
Class 'Laseration': 122 → train=97, val=12, test=13
Class 'Normal': 200 → train=160, val=20, test=20
Class 'Pressure Wounds': 602 → train=481, val=60, test=61
Class 'Surgical Wounds': 420 → train=336, val=42, test=42
Class 'Venous Wounds': 494 → train=395, val=49, test=50
Class 'healthy': 42 → train=33, val=4, test=5

✅ Done. Split saved to: /content/data_split


In [ ]:
# ✅ Train EfficientNetB0 with class weights and fine-tuning
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
import numpy as np
from collections import Counter
import os

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_PATH = "/content/data_split"
EPOCHS_HEAD = 10
EPOCHS_FINE = 5
LABEL_SMOOTH = 0.1

In [ ]:
# ✅ Load datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_PATH, "train"), image_size=IMG_SIZE,
    batch_size=BATCH_SIZE, label_mode="int", shuffle=True)
val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_PATH, "val"), image_size=IMG_SIZE,
    batch_size=BATCH_SIZE, label_mode="int", shuffle=False)
test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_PATH, "test"), image_size=IMG_SIZE,
    batch_size=BATCH_SIZE, label_mode="int", shuffle=False)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print("Classes:", class_names)


Found 2382 files belonging to 11 classes.
Found 296 files belonging to 11 classes.
Found 304 files belonging to 11 classes.
Classes: ['Abrasions', 'Bruises', 'Burns', 'Cut', 'Diabetic Wounds', 'Laseration', 'Normal', 'Pressure Wounds', 'Surgical Wounds', 'Venous Wounds', 'healthy']


In [ ]:
# ✅ Augmentation + Preprocessing
aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.2)
])

def prep(ds, augm=False):
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32), y))
    if augm:
        ds = ds.map(lambda x, y: (aug(x), y))
    ds = ds.map(lambda x, y: (preprocess_input(x), y))
    return ds.prefetch(tf.data.AUTOTUNE)

train_ds = prep(train_ds, True)
val_ds = prep(val_ds)
test_ds = prep(test_ds)


In [ ]:
# ✅ Class weights
labs = []
for _, y in tf.keras.utils.image_dataset_from_directory(
        os.path.join(DATA_PATH, "train"), image_size=IMG_SIZE,
        batch_size=1000, shuffle=False).unbatch().batch(1000):
    labs += list(y.numpy())
counts = Counter(labs)
total = sum(counts.values())
weights = {i: total / (NUM_CLASSES * counts[i]) for i in range(NUM_CLASSES)}
print("Class weights:", weights)


Found 2382 files belonging to 11 classes.
Class weights: {0: 1.6530187369882026, 1: 1.1219971738106453, 2: 2.023789294817332, 3: 2.706818181818182, 4: 0.5868440502586844, 5: 2.232427366447985, 6: 1.353409090909091, 7: 0.4501984501984502, 8: 0.6444805194805194, 9: 0.5482163406214039, 10: 6.56198347107438}


In [ ]:
!pip install -q --upgrade tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 75.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.


In [ ]:
# ✅ Build and compile model
base = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base.trainable = False
inp = layers.Input(shape=(224, 224, 3))
x = base(inp, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
out = layers.Dense(NUM_CLASSES, activation="softmax")(x)
model = tf.keras.Model(inp, out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)


In [ ]:
# ✅ Callbacks
cb = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=2),
    tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True)
]


In [ ]:
# ✅ Train head
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_HEAD, class_weight=weights, callbacks=cb)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 244s 3s/step - accuracy: 0.8327 - loss: 0.4231 - val_accuracy: 0.8074 - val_loss: 0.5895 - learning_rate: 0.0010
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 245s 3s/step - accuracy: 0.8377 - loss: 0.3980 - val_accuracy: 0.8041 - val_loss: 0.5866 - learning_rate: 0.0010
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 268s 4s/step - accuracy: 0.8347 - loss: 0.4008 - val_accuracy: 0.8041 - val_loss: 0.5717 - learning_rate: 0.0010
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 270s 4s/step - accuracy: 0.8481 - loss: 0.3831 - val_accuracy: 0.8142 - val_loss: 0.5493 - learning_rate: 0.0010
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.8573 - loss: 0.3698 - val_accuracy: 0.8176 - val_loss: 0.5412 - learning_rate: 0.0010
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 257s 3s/step - accuracy: 0.8462 - loss: 0.3761 - val_accuracy: 0.8209 - val_loss: 0.5334 - learning_rate: 0.0010
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 261s 3s/step - accuracy: 0.8592 - loss: 0.3278 - val_acc

In [ ]:
# ✅ Fine-tune last layers
base.trainable = True
for l in base.layers[:-20]: l.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=model.loss, metrics=["accuracy"])
model.fit(train_ds, validation_data=val_ds, epochs=10, class_weight=weights, callbacks=cb)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 307s 4s/step - accuracy: 0.7685 - loss: 0.5790 - val_accuracy: 0.8716 - val_loss: 0.4170 - learning_rate: 0.0010
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 299s 4s/step - accuracy: 0.8811 - loss: 0.2603 - val_accuracy: 0.9122 - val_loss: 0.3040 - learning_rate: 0.0010
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 264s 4s/step - accuracy: 0.9351 - loss: 0.1351 - val_accuracy: 0.8919 - val_loss: 0.3398 - learning_rate: 0.0010
Epoch 4/10
 6/75 ━━━━━━━━━━━━━━━━━━━━ 4:00 3s/step - accuracy: 0.9029 - loss: 0.1606

KeyboardInterrupt: 

In [ ]:
# ✅ Evaluate on test set
loss, acc = model.evaluate(test_ds)
print(f"\n✅ Test Accuracy: {acc:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.9275 - loss: 0.2206

✅ Test Accuracy: 0.8947


In [ ]:
model.save("/content/drive/MyDrive/wound_classifier_final.keras")
print("✅ Model saved to Google Drive!")

✅ Model saved to Google Drive!
